# 動的メモリ管理とスマートポインタ


----

## 1 　３種類のメモリ管理

----


### 1.1 メモリの管理方法について

これまで何度か「変数はメモ帳に書かれたメモ」という例え話をしてきました。<br>
実は、コンピュータがメモを扱う方法は、次の3種類に分けられます。

* 自動的な管理
* 静的(せいてき)な管理
* 動的(どうてき)な管理


### 1.2 自動的な管理

「自動的な管理」は、関数の中で宣言する変数の扱い方です。<br>
宣言した時点でメモ帳に書かれ、スコープが終了するとメモ帳から消されます。

この種類の変数は「ローカル変数」と呼ばれます。<br>
次のプログラムで使われている変数は、すべてローカル変数です。

**コード**

```cpp
#include <iostream>
#include <string>
using namespace std;

double f(int a) { // aはローカル変数
  double x = a * 2; // xはローカル変数
  return x;
}

int main() {
  int y = f(10); // yはローカル変数
  cout << y << endl;

  string s = "123"; // sはローカル変数
  for (int i = 0; i < (int)s.size(); i++) { // iはローカル変数
    cout << s[i] << " ";
  }
}
```

同じスコープに属するローカル変数は、宣言とは逆順に削除されます。<br>
例えば、上記のプログラムの場合、同じスコープに属する`y`と`s`は、`s`→`y`の順で削除されます。


### 1.3 静的(せいてき)な管理

「静的な管理」は、関数の外で定義する、または`static`(スタティック)キーワードを付けて宣言する変数の扱い方です。<br>
プログラムの実行直後、`main`関数が始まるより前にメモが作られ、プログラムが終了するまで消えません。

この種類の変数は「スタティック変数」と呼ばれます。<br>
さらに、関数の外で宣言したスタティック変数は、特別に「グローバル変数」と呼ばれます。

スタティック変数は安全に使うことが難しいため、多用してはいけません。

次のプログラムは、スタティック変数を使った例です。

**コード**

```cpp
#include <iostream>
#include <string>
using namespace std;

int x = 3; // xはグローバル変数(スタティック変数)
string s = "abc"; // sはグローバル変数(スタティック変数)

int f(int a) {
  static int y = 0; // yはスタティック変数
  y += a;
  return y;
}

int main() {
  cout << f(x) << endl;
  cout << f(5) << endl;
  cout << s << endl;
}
```

**実行結果**

```txt
3
8
abc
```

関数`f`の2回目の呼び出しで、戻り値が`5`ではなく`8`になっている点に注目してください。<br>
スタティック変数は関数が終了しても削除されないので、最後に関数を実行したときの値が残っているのです。

スタティック変数は、宣言とは逆順に削除されます。<br>
例えば、上記のプログラムのスタティック変数`x`と`s`は、`s`→`x`の順で削除されます。

>`static`(スタティック)は「長い間動かず、変化せず、その場に留まる。静的。」という意味です。<br>
>`global`(グローバル)は「全世界。世界的な」という意味です。プログラミング用語では「プログラムのどこからでも見える、使える」ことを意味します。


### 1.4 動的(どうてき)な管理

「動的な管理」では、プログラムからの指示で、好きなときにメモを作ったり消したりできます。<br>
メモがどこに作られるかはコンピュータまかせです。<br>
プログラムからは「メモの位置」、つまりポインタや参照を使ってメモを読み書きします。

動的な管理で作成されたデータは、「オブジェクト」などと呼ばれます。

動的な管理では、`new`キーワードや`delete`キーワードを使って確保したメモリに、データを保存します。<br>
データの読み書きには、メモリアドレスを記録したポインタを使います。



```cpp
#include <iostream>
#include <string>
using namespace std;

int* f(int a) {
  int* p = new int(a); // int型のオブジェクトを作成
  return p; // オブジェクトのアドレスを返す
}

int main() {
  int* x = f(1); // オブジェクトのアドレスを受け取る
  string* y = new string("23"); // string型のオブジェクトを作成

  cout << *x << *y << endl;

  delete y; // yが指すアドレスにあるオブジェクトを削除
  delete x; // xが指すアドレスにあるオブジェクトを削除
}
```

**実行結果**

```txt
123
```

`new`によって作成されたデータは、`delete`されるまで消えることはありません。<br>
そのため、スコープを超えてデータをやり取りできます。

自動的な管理や静的な管理では、データの削除はコンピュータが管理しました。<br>
ですが、動的な管理では、プログラマが責任を持ってデータを削除(`delete`を使うなど)しなくてはなりません。


----

## 2 安全でない動的メモリ管理

----


### 2.1 動的メモリ管理を使う理由

プログラムから利用できるメモリは、コンピュータに搭載されている物理的な「メモリ」そのものです。<br>
コンピュータのOSがプログラムを実行するとき、「物理的なメモリ」の一部をプログラム用に割り当てます。<br>
そして、プログラムは、割り当てられたメモリの範囲内で、必要な処理を実行します。

以下の図は、OSから割り当てられたメモリの概念図です。

$$
\boxed{プログラム本体}\boxed{スタティック}\boxed{　　　　ヒープ　　　　}\boxed{スタック}
$$

* プログラム本体: プログラム自身が読み込まれるメモリ領域です。
* スタティック: 「静的メモリ管理」用のメモリ領域です。
* ヒープ: 「動的メモリ管理」用のメモリ領域です。
* スタック: 「自動的メモリ管理」用のメモリ領域です。

一般的には、「ヒープ」が最大のメモリ容量を持ちます。柔軟なメモリ管理ができるのは、動的メモリ管理だけだからです。

小規模なプログラムであれば、「自動的な管理」と「静的な管理」に必要なだけのメモリがあれば、十分に処理を完了させられます。

しかし、ある程度以上の規模を持つプログラムでは、割り当てられたメモリが、すべての処理を行うために必要なメモリ量より少ない、という事態が発生します。

>例えば、2025年現在のゲーム機のメモリ容量はおよそ4GB～32GBです。しかし、ゲームアプリの容量は数百GBに達することもあり、とてもすべてをメモリに読み込むことはできません。

「動的メモリ管理」を使うと、「ある処理を実行中だけ必要なメモリ」を作成し、必要がなくなったら「他の処理のために削除」できるようになります。

C++には、標準で以下の2つの動的メモリ管理が組み込まれています。

* `new`(ニュー)キーワードと`delete`キーワードを使ったメモリ管理
* `malloc`(マロック)関数と`free`(フリー)関数を使ったメモリ管理

ただし、これらの基本機能は安全性に問題があるため、2025年現在では直接使われことは少なくなっています。<br>
しかし、あとで説明する「安全な動的メモリ管理」でも、内部的にはこれらの機能を使っています。

また、これらの機能が使われているプログラムを見たり、利用したりすることもあるでしょう。<br>
そこで、この章では、基本的なメモリ管理の使いかたを解説します。


### 2.2 newとdelete

`new`(ニュー)と`delete`(デリート)は、C++言語の最も基本的な動的メモリ管理の仕組みです。

動的メモリ管理では、実際には「ヒープ」と呼ばれるメモリ領域を管理します。

プログラムが起動すると、ヒープメモリ全体が「未使用」状態に設定されます。<br>
`new`が呼ばれると、ヒープメモリの一部を「使用中」に設定し、そのアドレスを返します。<br>
`delete`が呼ばれると、アドレスで指定されたメモリを「未使用」に戻します。

`new`キーワードの書式は次のようになっています。

>**書式**
>
>```cpp
>new 型;
>new 型(初期値);
>new 型[要素数];
>```
>
>**戻り値**
>
>最初の2つの形式では、指定された型を格納するためのメモリを確保し、型のコンストラクタを呼び出し、確保したメモリのアドレスを返します。
>
>`new 型[要素数]`の形式では、「要素数」で指定されたサイズの配列を格納するメモリを確保し、すべての要素にコンストラクタを呼び出して、配列の先頭アドレスを返します。この形式では初期値を指定できません。
>
>どの形式でも、戻り値の型は「指定した型のポインタ型」となります(例: `new int`の場合`int*`)。

`delete`キーワードの書式は次のようになっています。

>**書式**
>
>```cpp
>delete ポインタ変数;
>delete[] ポインタ変数;
>```
>
>**戻り値**
>
>ポインタ変数が指すオブジェクトのデストラクタを実行し、メモリを開放して他のプログラムから使えるようにします。<br>
>`delete[]`形式は配列用です。配列ではない変数に`delete[]`を使ったり、逆に配列変数に`delete`を使ってしまうと、論理エラーや実行時エラーが起こりります。

次のプログラムは、`new`と`delete`の使用例です。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```

`new`と`delete`をつかうとき、最も重要なことは「必ず`delete`を実行する」ことです。<br>
動的な管理で作成されたオブジェクトは、`delete`を実行しない限り削除されないからです。<br>
もし削除し忘れると、オブジェクトが占めていたメモリ領域は「使用中」のままになり、プログラムが終了するまで二度と使えなくなってしまうからです。<br>
この状態は「メモリ・リーク(メモリ漏れ)」と呼ばれています。


#### メモリ・リーク

コンピュータに搭載されているメモリは有限なので、メモリ・リークが頻発すると、コンピュータ全体で使用できるメモリがどんどん減っていきます。
その結果、徐々にコンピュータの実行効率が低下し、最終的には実行時エラーを起こしてアプリが強制終了します。

また、ポインタは「どこかの変数を参照している」状態と、「`new`で作られたオブジェクトを参照している」状態を区別できません。

そのため、プログラマが「このポインタは`new`で作ったやつを参照してるから、`delete`しなくちゃ」という判断をしなくてはなりません。<br>
もし判断を間違えると「メモリ・リーク」が起こります。


#### deleteしたメモリを読み書きしない

`delete`によって削除したあとのオブジェクトを読み書きしないように、注意する必要があります。<br>
もし削除したオブジェクトを読み書きしようとすると「実行時エラー」や「論理エラー」が発生します。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除

  // 削除したオブジェクトのデータを読もうとすると…
  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
83096 0 -1910714224
```


#### 多重deleteをしない

同じオブジェクトに`delete`を2回以上実行してしまうと、2回目の`delete`で「実行時エラー」が発生します。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除
  delete p; // 削除済みのオブジェクトを再び削除しようとすると…
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
デストラクタを実行
free(): double free detected in tcache 2
```

この多重`delete`問題に限れば、ポインタ変数に`nullptr`を代入することで回避できます(`delete`は`nullptr`に対して何もしないため)。<br>
ただし、`nullptr`の代入で防げるのは多重`delete`だけです。他の問題は防げません。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  Enemy* p = new Enemy(6, 4, 3); // Enemyオブジェクトを作成

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  delete p; // Enemyオブジェクトを削除
  p = nullptr; // 削除後に nullptr を代入
  delete p; // p は nullptr なので何も起こらない
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```


### 2.3 vectorと動的メモリ管理

動的メモリ管理で作成したポインタを、`vector`配列に設定する場合、データを削除する前に`delete`を実行しなくてはなりません。

**コード**

```cpp
#include <iostream>
#include <vector>
using namespace std;

int main() {
  vector<int*> v(5); // int*の配列

  // newを使ってメモリを確保
  for (int i = 0; i < 5; i++) {
    v[i] = new int(i + 1);
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << **i << " "; // イテレータとポインタで、'*'記号が2つ必要
  }
  cout << endl;

  //v.pop_back(); // NG: メモリ・リーク
  //v.claer(); // NG: メモリ・リーク

  // OK
  delete v.back();
  v.pop_back();

  // OK
  for (auto i = v.begin() + 2; i != v.end(); i++) {
    delete *i;
  }
  v.clear();
}
```

`delete`せずにポインタ変数を削除してしまうと、メモリ・リークが起こります。<br>
これは汎用アルゴリズムでも同様です。`remove`関数など、削除が起こりうる関数はすべて使えないと考えてください。


### 2.3 mallocとfree

`malloc`(マロック)関数と`free`(フリー)関数は、C言語の動的メモリ管理の仕組みです。<br>
C++ではほとんど使われないため、最低限の解説だけを行います。

`malloc`関数の書式は次のようになっています。

>**書式**
>
>```cpp
>void* malloc(size_t size);
>```
>
>**引数**
>
>* size&emsp;確保するメモリのバイト数
>
>**戻り値**
>
>少なくとも`size`バイトのメモリ領域を確保し、確保したメモリ領域の先頭アドレスを返します。<br>
>戻り値の型は`void*`なので、必要な型にキャストして使います。

`free`関数の書式は次のようになっています。

>**書式**
>
>```cpp
>void free(void* p);
>```
>
>**引数**
>
>* p&emsp;解放するメモリアドレス
>
> 引数`p`が指すアドレスのメモリ領域を解放し、他のプログラムから使えるようにします。

`new`と`delete`と比較したとき、最大の違いは「コンストラクタもデストラクタも実行しない」ことです。<br>
そのため、コンストラクタやデストラクタを持つ構造体やクラスを`malloc`で作成しようとすると、かなり面倒なことになります。

`int`などの組み込み型や、メンバ変数だけの構造体、クラスの場合は、`malloc`と`free`を使っても問題もありません。

**コード**

```cpp
#include <iostream>
using namespace std;

struct Enemy {
  int hp, power, armor;
};

int main() {
  Enemy* p = (Enemy*)malloc(sizeof(Enemy));
  p->hp = 6;
  p->power = 4;
  p->armor = 3;

  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;

  free(p);
}
```

**実行結果**

```txt
6 4 3
```

`malloc`関数で確保したメモリは`free`関数で解放しなくてはなりません。<br>
`new`と`delete`の場合と同様に、解放し忘れると「メモリ・リーク」が起こります。<br>
削除したオブジェクトを読み書きすると「実行時エラー」や「論理エラー」が発生するのも同様です。

また、あるポインタが`new`で作られたのか、`malloc`で作られたのかを区別する方法はありません。<br>
そのため、プログラマが「このポインタは`malloc`で作ったやつだから、`free`を使わなくちゃ」という判断をする必要があります。<br>
もし判断を間違えると「論理エラー」や「実行時エラー」が発生します。


----

## 3 安全な動的メモリ管理

----


### 3.1 スマート・ポインタ


#### new, deleteの問題点

`new`と`delete`を使う場合、問題のひとつは「`delete`のタイミングをプログラマが適切に管理する必要がある」ということでした。<br>
そのため、うっかり必要なオブジェクトを`delete`してしまったり、些細な勘違いで`delete`を書き忘れてしまう危険性があります。

これらの問題は、コンピュータが管理する「自動的なメモリ管理」や「静的なメモリ管理」では起こりません。<br>
もし「動的なメモリ管理」でも、コンピュータに削除のタイミングを任せられるなら、問題の多くが解決されると期待できます。


#### スマートポインタで問題を解決

そこで、「スマート・ポインタ」が開発されました。<br>
スマートポインタは、オブジェクトを削除するタイミングを、コンピュータが管理できるように設計されたクラスです。<br>
そのため、プログラマは削除のタイミングを考える必要はありません。

C++には、次に示す3種類のスマート・ポインタがあります。

* `shared_ptr`(シェアード・ポインタ): 標準的なスマートポインタ
* `unique_ptr`(ユニーク・ポインタ): オブジェクトを指すポインタがひとつだけのスマートポインタ
* `weak_ptr`(ウィーク・ポインタ): 「循環参照(じゅんかん・さんしょう)」という問題を解決するためのスマートポインタ

これらのスマートポインタに共通する仕組みは、「デストラクタでメモリを解放する」というものです。<br>
デストラクタは、変数を削除するときに自動的に呼び出されるので、変数が削除されると、自動的にメモリも解放されるわけです。

この章では、基本となる`shared_ptr`だけ解説します。<br>
`unique_ptr`と`weak_ptr`については、`cpprefjp.github.io`などのリファレンスサイトを見てください。


### 3.2 shared_ptr(シェアード・ポインタ)

`shared_ptr`クラスは、C++の標準的なスマートポインタです。

`shared_ptr`を使うには`<memory>`ヘッダをインクルードします。<br>
`shared_ptr`は、テンプレートを使って次のように宣言されています。

```cpp
template<typename T>
class shared_ptr;
```

動的メモリ管理される「オブジェクト」を作るには、`make_shared`(メイク・シェアード)関数を使います。

>**書式**
>
>```cpp
>template<typename T>
>shared_ptr make_shared<T>(コンストラクタに渡す引数);
>```
>
>**テンプレート引数**
>
>T&emsp;作成する型の名前
>
>**引数**
>
>Tに指定した型のコンストラクタに渡す引数です。<br>
>内容はコンストラクタによって異なります。
>
>**戻り値**
>
>確保したメモリアドレスを管理する`shared_ptr`型の値を返します。

`make_shared`関数は、次の処理を行います。

1. 指定された「データの型」を記録するためのメモリ領域を、コンピュータの自由に使えるメモリ領域から確保する
2. 確保したメモリ領域に対して「データの型」のコンストラクタを実行する
3. 確保したメモリを管理する`shared_ptr`型のデータを返す。

そして、`shared_ptr`型の変数が削除されると、次の処理が順番に実行されます。

1. 「データの型」のデストラクタを実行する
2. 参照しているメモリを開放し、他のプログラムから使えるようにする

>より正確には、上記の処理が行われるのは「同じメモリを参照しているすべての`shared_ptr`が削除されたとき」です。

次のプログラムは、`shared_ptr`型を使った例です。

**コード**

```cpp
#include <iostream>
#include <memory>
using namespace std;

struct Enemy {
  Enemy(int h, int p, int a) : hp(h), power(p), armor(a) { cout << "コンストラクタを実行" << endl; }
  ~Enemy() { cout << "デストラクタを実行" << endl; }

  int hp, power, armor;
};

int main() {
  auto p = make_shared<Enemy>(6, 4, 3);
  cout << p->hp << ' ' << p->power << ' ' << p->armor << endl;
}
```

**実行結果**

```txt
コンストラクタを実行
6 4 3
デストラクタを実行
```

プログラムでは、返された「アドレス」を`shared_ptr`変数に代入し、ポインタを使ってデータを読み書きします。<br>
作成方法こそ普通のポインタとは異なりますが、使いかたは普通のポインタと同じです。


#### 変数をコピーしたときの挙動

普通のポインタ変数をコピーすると、コピー先のポインタ変数もおなじメモリアドレスを指します。<br>
次のプログラムは、ポインタ変数のコピーの動作を示す例です。

**コード**

```cpp
#include <iostream>
using namespace std;

int main() {
  int* a = new int(10);

  int* b = a; // aとbは同じメモリアドレスを指す

  *b += 5;
  cout << *a << endl; // 15が出力される

  delete b; // OK: aとbは同じメモリアドレスを指している
            //     bを通じてメモリを解放したので、aはdeleteする必要はない
}
```

このくらい短いプログラムであれば、ポインタ変数`a`と`b`のどちらか片方を`delete`すれば、もう片方を`delete`する必要はないし、してはいけない、ということを覚えていられるでしょう。

ですが、もっと長いプログラムで、さまざまな場所にポインタ変数のコピーがある、という状況では、「正しく1度だけ`delete`する」というのは非常に困難になります。

この問題も、`shared_ptr`を使えば解決できます。`shared_ptr`は「コピーされた回数」を覚える機能を持っているからです。

次のプログラムは、`shared_ptr`を使って上のプログラムを書き直した例です。

**コード**

```cpp
#include <iostream>
#include <memory>
using namespace std;

int main() {
  shared_ptr<int> a = make_shared<int>(10);

  shared_ptr<int> b = a; // aとbは同じメモリアドレスを指す
                         // aはコピーされたことを記録する
  *b += 5;
  cout << *a << endl; // 15が出力される
} // ここでaとbが削除され、メモリの解放が1度だけ行われる
```

このように、`shared_ptr`を使うと、ポインタのコピーの管理とメモリの解放を、全てコンピュータに任せられます。<br>
どれだけコピーを作ろうと、頭を悩ませて、いつ、どこに`delete`を書くべきかを考える必要はありません。


### 3.3 vectorとshared_ptr

`new`で取得したポインタを`vector`配列に格納する場合、削除する前に`delete`しなくてはならない、という説明をしました。<br>
ですが、`shared_ptr`を使えば、削除の心配はなくなります。

また、汎用アルゴリズムも使えるようになります。<br>
というのも、汎用アルゴリズムで削除された場合でも、必ずデストラクタが呼び出されるからです。

次のプログラムは、`vector`と動的メモリ管理を組み合わせたプログラムを、`shared_ptr`を使うように修正したものです。

**コード**

```cpp
#include <iostream>
#include <vector>
#include <memory>
using namespace std;

int main() {
  vector<shared_ptr<int>> v(5); // shared_ptr<int>の配列

  // make_sharedを使ってメモリを確保
  for (int i = 0; i < 5; i++) {
    v[i] = make_shared<int>(i + 1);
  }

  for (auto i = v.begin(); i != v.end(); i++) {
    cout << **i << " "; // イテレータとポインタで、'*'記号が2つ必要
  }
  cout << endl;

  v.pop_back(); // OK
  v.clear(); // OK

  // NG: shared_ptrが管理しているメモリを、勝手にdeleteしてはいけない
  //delete v.back();
  //v.pop_back();

  // NG: shared_ptrが管理しているメモリを、勝手にdeleteしてはいけない
  //for (auto i = v.begin(); i != v.end(); i++) {
  //  delete *i;
  //}
  //v.clear();
}
```

`delete`がなくなり、`pop_back`や`clear`メンバ関数でいつでも要素を削除できるようになっています。<br>
プログラムに登場してはいませんが、もちろん`erase`も使えます。



### 3.4 安全な動的メモリ管理を使おう

ここまで説明したように、`new`, `delete`または`malloc`, `free`による動的メモリ管理は、`shared_ptr`と比べて以下の問題があります。

* ポインタの参照先が普通の変数なのか、動的メモリ管理で作られたものなのかの判断が難しい
* 削除を忘れると「メモリ・リーク」が発生する
* 削除したオブジェクトを読み書きすると「実行時エラー」や「論理エラー」が発生する
* `vector`と組み合わせる場合、要素を削除する前に`delete`が必要になる
* 要素の削除を伴う汎用アルゴリズムが使えない

多くのプログラム書いて経験を積むまでは、これらは大した問題ではないように思えるかもしれません。<br>
それでも、特に理由がないかぎり、動的メモリ管理には`shared_ptr`クラスと`make_shared`関数を使うことをおすすめします。


----

## 4 練習問題

----

以下の手順にしたがって、各問題のプログラムを完成させなさい。

1. `%%writefile ...`の下の行からがプログラムです。問題文に従ってプログラムを修正、または追加してください。
2. プログラムを修正したら、セルの右側にある`▶`をクリックします。すると、ファイルが保存されます。
3. 「動作テスト」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、実行結果が表示されます。<br>
   このセルは、修正したプログラムの動作を確認するために使ってください。
4. 「実行」セルの`▶`をクリックすると、2で保存したファイルがコンパイル＆実行され、結果の成否が判定されます。
5. 判定に成功したら`AC`と表示されます。次の問題に進んでください。
6. 失敗したら`WA`と表示されます(その前にエラーメッセージが表示される場合もあります)。<br>
   これは、プログラムのどこかにエラーがあることを意味します。<br>
   「動作テスト」を使ってエラーを修正し、`AC`を目指してください。


In [ ]:
# 自動メモリ管理と静的メモリ管理で各1問
# new, deleteで2問
# malloc, freeで1問
# shared_ptrで3問
# 応用問題を2問

### ❓️問題１ 自動的なメモリ管理

自動的なメモリ管理によって、変数が削除される順番を確認したいです。<br>
数字が`1`, `2`, `3`, `4`の順番で出力されるように、変数`a`, `b`, `c`, `d`のコンストラクタ引数に`1`～`4`のいずれかの数字を入れなさい。

**出力例**

```txt
1
2
3
4
```


In [ ]:
%%writefile practice_01a.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

int main() {

  // 以下の4つの？を、1～4のいずれかの数字に書き換える
  S a(？);
  {
    S b(？);
    {
      S c(？);
    }
    S d(？);
  }
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && ./practice_01a

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n2\n3\n4") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01a practice_01a.cpp && ./practice_01a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01a.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

int main() {

  // 以下の4つの？を、1～4のいずれかの数字に書き換える
  S a(4);
  {
    S b(3);
    {
      S c(1);
    }
    S d(2);
  }
}

### ❓️問題２ 静的なメモリ管理

自動的なメモリ管理に加えて、静的なメモリ管理によって、変数が削除される順番を確認したいです。<br>
数字が`1`, `2`, `3`, `4`, `5`の順番で出力されるように、変数`a`, `b`, `c`, `d`, `e`のコンストラクタ引数に`1`～`5`のいずれかの数字を入れなさい。

**出力例**

```txt
1
2
3
4
5
```


In [ ]:
%%writefile practice_01b.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

// 以下の5つの？を、1～5のいずれかの数字に書き換える
S e(？);

int main() {

  S a(？);
  {
    S b(？);
    {
      static S c(？);
    }
    static S d(？);
  }
}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && ./practice_01b

In [ ]:
# @title 実行
!diff -Z <(echo -e "1\n2\n3\n4\n5") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_01b practice_01b.cpp && ./practice_01b) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_01b.cpp
#include <iostream>
using namespace std;

struct S {
  S(int n) : x(n) {} // コンストラクタ
  ~S() { cout << x << endl; } // デストラクタ

  int x = 0;
};

// 以下の5つの？を、1～5のいずれかの数字に書き換える
S e(5);

int main() {

  S a(2);
  {
    S b(1);
    {
      static S c(4);
    }
    static S d(3);
  }
}

### ❓️問題３


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題４


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題５


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題６


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題７


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題８


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題９


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

### ❓️問題１０


<br><details><summary>🗝️(クリックでヒントを見る)</summary>

**プログラム例**

1.
2.
3.

</details><br>

**入力データ例（１）**

```txt

```

**出力例（１）**

```txt

```

**入力データ例（２）**

```txt

```

**出力例（２）**

```txt

```


In [ ]:
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}

In [ ]:
# @title 動作テスト
!g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "この下をクリックして、を入力" && ./practice_02a

In [ ]:
# @title 実行
!diff -Z <(echo -e "") <(g++ -std=c++20 -O2 -Wall -Wextra -o practice_02a practice_02a.cpp && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a && echo "" | ./practice_02a) > nil && test $? -eq 0 && echo -e "\033[32;1mAC" || echo -e "\033[31;1mWA"

In [ ]:
# @title 🔒解答例(どうしても問題を解けない場合に見てください)
%%writefile practice_02a.cpp
#include <iostream>
#include <string>
#include <stdlib.h>
using namespace std;

int main() {
  // この下に、

}